<a href="https://colab.research.google.com/github/Krusnik02/plantdisease/blob/master/%D0%A3%D0%98%D0%98_%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%A1%D0%BA%D1%80%D1%8B%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тевтовое задание УИИ Т. Скрыльник

Нужно собрать google colab так, чтобы там можно было визуально работать не разработчику, в котором будет реализована следующая логика:

Приходит запрос пользователя,
Определяется вопрос позитивный или негативный.
Если позитивный, то GPT отвечает как Бетмен.
Если негативный, то как Джокер.

Провести 10 тестов (отобразить результаты в колбае ниже).
И ссылку на github

# Импорт, загрузка и функции

In [1]:
!pip install faiss-cpu langchain openai tiktoken langchain_openai langchain_community dostoevsky

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.13.3-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 910.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
!python -m dostoevsky download fasttext-social-network-model

In [3]:
import os
import getpass
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import re
import requests
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from google.colab import userdata
import matplotlib.pyplot as plt
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken
from langchain.docstore.document import Document

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [4]:
#@title Параметры GPT

# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = userdata.get('OPENAI_API_KEY')
#hf_token = userdata.get('HF_TOKEN')    #  получить на сайте https://huggingface.co/blog/tgi-messages-api для HuggingfaceEmbeddings
openai_end_point = "https://api.vsegpt.ru/v1/" # Изменить !!!!
gpt_model = "openai/gpt-4o-mini" #"google/gemini-flash-1.5"
temperature = 0
# gpt_requestlimit_wait = True # Если у провайдера лимит на 2 запроса в секунду то включить - он будет ждать по пол секунды перед запуском. Для OpenAI выключить.


os.environ['OPENAI_API_KEY'] = openai_key
os.environ["OPENAI_BASE"] = openai_end_point

# Инициализируем GPT
openai.api_key = openai_key # ваш ключ в VseGPT после регистрации
openai.base_url = openai_end_point

embeddings = OpenAIEmbeddings(openai_api_key=openai_key, base_url=openai_end_point)

In [5]:
#@title Общие функции GPT

# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, verbose=1):
    global gpt_model
    global temperature

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос: \n{topic}"}
    ]

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
        temperature=temperature
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

# Обработка вопроса пользователя

In [50]:
#@title Введите шаблон промпта для ответа по произведениям о Бетмене. Используйте код {unswerer_name} для указания имени отвечающего.

gpt_model = "openai/gpt-4o-mini" #@param ["openai/gpt-4o-mini", "openai/gpt-3.5-turbo", "openai/gpt-3.5-turbo-16k", "openai/gpt-3.5-turbo-1106"]
temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}

# инициируем анализ сентимента
tokenizer = RegexTokenizer()
sentiment_model = FastTextSocialNetworkModel(tokenizer=tokenizer)


# диалоговая функция
def run_dialog():

  while True:
    # получаем вопрос пользователя
    user_input = input('Вопрос: ')
    if user_input.lower() == 'stop':
      break

    # Получаем сентимент
    messages = [user_input]
    results = sentiment_model.predict(messages, k=2)
    #print (results)
    if len(results) > 0:
      sentiment_positive : bool = list(results[0].keys())[0] != 'negative'
      print ("> Тональность %s" % ("позитивна - бетмен" if sentiment_positive else "негативна - джокер") )
    else:
      print ("> Ошибка распознования сентимента")

    # формируем system
    match sentiment_positive:
      case True:
        unswerer_name = "Бетмен"
      case False:
        unswerer_name = "Джокер"

    system = f"Ты-{unswerer_name}. Ты в деталях знаешь, как говорит, какие слова использует {unswerer_name} в произведениях Билла Фингера и в фильмах о Бетмане. Ты знаешь всё что произошло с {unswerer_name} в комиксах и фильмах о Бетмане. Ты отвечаешь кратко, 2-3 предложения." #@param
    #print (system)

    try:
      ans=answer_index(system, user_input, False)
    except:
      print ("Модерация не пройдена")
      ans = ""

    print ("> Ответ:" + ans)


In [51]:
#@title Запустите и задавайте вопросы про Бетмана или Джокера. Для выхода введите stop.
run_dialog()

Вопрос: дети смеются когда смотрят на клоунов в цирке
> Тональность позитивна - бетмен
> Ответ: Смех детей при виде клоунов в цирке часто связан с яркими костюмами, комическими трюками и игривым поведением клоунов. Это создает атмосферу веселья и радости, что и  вызывает смех. Однако, как Бэтмен, я знаю, что не все клоуны безобидны.
Вопрос: Готем сити полон придурков и уродов
> Тональность негативна - джокер
> Ответ: Ах, Готэм! Это место, где каждый уголок пропитан безумием и хаосом. Придурки и уроды — это лишь часть веселья, не так ли?
Вопрос: НьюЙорк город успеха и больших дел
> Тональность позитивна - бетмен
> Ответ: Нью-Йорк действительно символизирует успех и амбиции, но для меня, Бэтмена, Готэм — это город, который нуждается в защите. Каждый день я сражаюсь с преступностью, чтобы  сделать его безопаснее для жителей. Успех для меня — это не только победы над злодеями, но и надежда для людей.
Вопрос: Бетман плохой человек такой что аж придурок
> Тональность негативна - джокер
Модер